In [1]:
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from models.CNN import create_cnn_model

In [2]:
def load(f):
    # Move up one directory when loading the data
    file_path = os.path.join('../../../', f)
    return np.load(file_path)['arr_0']

# Load the data
x_train = load('kmnist-train-imgs.npz')/ 255.0
x_test = load('kmnist-test-imgs.npz')/ 255.0
y_train = load('kmnist-train-labels.npz')
y_test = load('kmnist-test-labels.npz')
# Reshape the data for CNN input
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [3]:
input_shape = x_train.shape[1:]  # 784 for KMNIST
num_classes = y_train.max() + 1
x_train_aug, x_val, y_train_aug, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


In [4]:
hyperparameters = {
    'num_layers': 3,
    'filters': [32, 64, 64],
    'kernel_sizes': [(3, 3), (3, 3), (3, 3)],
    'activations': ['relu', 'relu', 'relu'],
    'dense_units': 64,
    'dense_activation': 'relu',
    'pooling': 'avg',
    'batch_normalization': True
}

In [5]:
best_model_noise = create_cnn_model(input_shape, num_classes, hyperparameters)
best_model_no_noise = create_cnn_model(input_shape, num_classes, hyperparameters)
datagen_noise = ImageDataGenerator(
    preprocessing_function=lambda x: np.clip(x + np.random.normal(0, 0.1, x.shape), 0, 1)  # Noise injection
)
datagen_no_noise = ImageDataGenerator()
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
datagen_val = ImageDataGenerator()
# Wrap the datagen with our custom generator
train_generator_noise = datagen_noise.flow(x_train_aug, y_train_aug, batch_size=32)
train_generator_no_noise = datagen_no_noise.flow(x_train_aug, y_train_aug, batch_size=32)
val_generator = datagen_val.flow(x_val, y_val, batch_size=32)
# Train the model with custom augmentation
history_noise = best_model_noise.fit(train_generator_noise,
                        epochs=20,
                        validation_data=val_generator,  
                        verbose=1,
                        callbacks=[early_stopping])
history_no_noise = best_model_no_noise.fit(train_generator_no_noise,
                        epochs=20,
                        validation_data=val_generator,
                        verbose=1,
                        callbacks=[early_stopping])

# Evaluate on the test set
test_loss_noise, test_accuracy_noise = best_model_noise.evaluate(x_test, y_test, verbose=0)
test_loss_no_noise, test_accuracy_no_noise = best_model_no_noise.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy with Noise Injection: {test_accuracy_noise:.4f}")
print(f"Test loss with Noise Injection: {test_loss_noise:.4f}")
print(f"Test accuracy without Noise Injection: {test_accuracy_no_noise:.4f}")
print(f"Test loss without Noise Injection: {test_loss_no_noise:.4f}")

Epoch 1/20
1688/1688 [==============================] - 16s 7ms/step - loss: 0.1925 - accuracy: 0.9488 - val_loss: 0.0775 - val_accuracy: 0.9753
Epoch 2/20
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0710 - accuracy: 0.9797 - val_loss: 0.0457 - val_accuracy: 0.9870
Epoch 3/20
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0489 - accuracy: 0.9851 - val_loss: 0.1010 - val_accuracy: 0.9700
Epoch 4/20
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0387 - accuracy: 0.9882 - val_loss: 0.0380 - val_accuracy: 0.9887
Epoch 5/20
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0330 - accuracy: 0.9901 - val_loss: 0.0405 - val_accuracy: 0.9875
Epoch 6/20
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0260 - accuracy: 0.9919 - val_loss: 0.0506 - val_accuracy: 0.9857
Epoch 7/20
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0220 - accuracy: 0.9931 - val_loss: 0.0569 - val_ac